In [1]:
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd
import dask.distributed
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pd.set_option('display.max_rows',10)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
gdf_osyu_100m_mesh = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/gdf_osyu_100m_mesh.shp').filter(['Meshcode','longitude', 'latitude',  'geometry'])

In [5]:
gdf_station = gpd.read_file('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/src/国土数値情報/鉄道/N02-22_Station.shp', encoding='shift-jis') #このデータは道路中心線

In [6]:
#座標系を変更
src_proj = 6668 # 変換前の座標系を指定
dst_proj = 6678  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_station.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_station = gdf_station.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [7]:
#線路のデータフレームのカラム名を変換
station_columns = {'N02_001': 'station_classification',
                   'N02_002': 'station_Business_Type',
                   'N02_003': 'station_route_name',
                   'N02_004':'station_operating_company',
                   'N02_005':	'station_name',
                   'N02_005c': 'station_code',
                   'N02_005g': 'station_group_code'}
gdf_station.rename(columns=station_columns, inplace=True)

In [8]:
#重心座標のPointを100mメッシュのデータフレームに追加
gdf_osyu_100m_mesh['center_geometry'] = [Point(x) for x in zip(gdf_osyu_100m_mesh['longitude'], gdf_osyu_100m_mesh['latitude'])]

In [9]:
# cKDTreeを使用して最近傍と距離を求める
line_coords = gdf_station.geometry.apply(lambda geom: list(geom.coords[0]))
point_coords = gdf_osyu_100m_mesh.center_geometry.apply(lambda geom: list(geom.coords[0]))

line_tree = cKDTree(line_coords.tolist())
point_tree = cKDTree(point_coords.tolist())

# 最近傍の検索
distances, line_indices = line_tree.query(point_coords.tolist(), k=1)
nearest_lines = gdf_station.iloc[line_indices]

# 距離を結果に追加
nearest_lines['distance_station'] = distances

# 結果を表示
print(nearest_lines)

#lineのgeometryxはいらないので削除
nearest_lines.drop('geometry', axis=1, inplace=True)

#100mメッシュに対して同じ高速道路が最近傍だとindexが複数できてしまうので並び替え
nearest_lines.reset_index(drop=True, inplace=True)

#100mメッシュのデータフレームに最近棒の高速道路を追加
gdf_station = gdf_osyu_100m_mesh.join(nearest_lines)

     station_classification station_Business_Type station_route_name  \
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
...                     ...                   ...                ...   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   
6635                     11                     2                花輪線   

     station_operating_company station_name station_code station_group_code  \
6635                   東日本旅客鉄道           兄畑       000710

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
<ipython-input-9-09bea65b125f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_lines.drop('geometry', axis=1, inplace=True)


In [10]:
#座標系を変更
src_proj = 6678 # 変換前の座標系を指定
dst_proj = 4326  # 変換後の座標系を指定
# ポイント（ダムデータ、TransformPointの引数は緯度,経度の順番で指定）
gdf_station.crs = f'epsg:{src_proj}'  # 変換前座標を指定
gdf_station = gdf_station.to_crs(epsg=dst_proj)  # 変換後座標に変換

In [11]:
# #geometryが二つあると出力できないのでcenter_geometryを削除
gdf_station.drop(columns=['longitude', 'latitude', 'geometry', 'center_geometry'], axis=1, inplace=True)

In [12]:
gdf_station.to_csv('/content/drive/MyDrive/水道管劣化予測/水道管劣化予測/result/奥州市 最近傍鉄道/奥州市_最近傍駅.csv', encoding='shift-jis')